# Docker for Robotics

## ROS2 Dev Container Setup
- Install docker engine for Ubuntu using the convenience script > Post-installation steps.
- Install remote development extension in VSCode.
- Install and configure nvidia container toolkit
- Disconnect VPN
- Don't install Python/Anaconda
- Install correct cuda for latest PyTorch (default cuda may be fine), install default Pytorch
```json
{
  "name": "humble desktop-full",
  "dockerFile": "Dockerfile",
  "runArgs": [
    "--privileged",
    "--network=host",
    "--gpus","all"
  ],
  "workspaceMount": "source=${localWorkspaceFolder},target=/${localWorkspaceFolderBasename},type=bind",
  "workspaceFolder": "/${localWorkspaceFolderBasename}",
  "mounts": [
    "source=${localEnv:HOME}${localEnv:USERPROFILE}/.bash_history,target=/home/vscode/.bash_history,type=bind"
],
"features": {
	"ghcr.io/devcontainers/features/desktop-lite:1": {},
  "ghcr.io/devcontainers/features/nvidia-cuda:1": {},
  "ghcr.io/raucha/devcontainer-features/pytorch:1": {}
}
}
```

**Running GUI**
xhost +local:  
In terminal, and rebuilding container.

# Making a mobile robot

## Introduction

Differential Drive: robot with two wheels driving it. 

Base link: main coordinate system: x-pointing forward, y-pointing left, z-pointing up.

![Robot State Publisher](./images/Robot_state_publisher.png)

The main coordinate frame of the robot is called `base_link` with x pointing forward, y pointing left, z pointing up.  

We start with config files written in URDF format that altogether describes the structure of the robot. These are combined using xacro into a single processed URDF file.  

This URDF robot structure is passed to `robot_state_publisher` file. This makes URDF data in `/robot_description` topic, and broadcasts appropriate transforms `/tf`.  

If any join moves, `robot_state_publisher` expectes the input to be published on the `/joint_states` topic.

While testing, we can use the `/joint_state_publisher_gui` to fake these values.

We run all of these files using launch files.

Everytime we add a URDF file, we need to run `colcon build --symlink-install` and source it using `source install/setup.bash`. After change, we need to quit and relaunch `/robot_state_publisher`. If Rviz doesn't show updates in URDF visualization, refresh.

### Adding files
You can copy `blue_bot` and `blue_bot_controller` folders to your `/src` folder and run `colcon build --symlink-install` and source it.

Run the `/robot_state_publisher` by running the launch file using
```bash
ros2 launch blue_bot rsp.launch.py
```


## Robot Model


### URDF syntax
URDF is based on XML, where everything is represented as a series of tags nested inside each other.

```XML
<?xml version="1.0"?>
<robot name="my_bot">
<link name="link_name"></link>
<joint name="joint_name"></joint>
<link></link>
...
</robot>
```


#### Creating URDF
In our `workspace/src/`, we have `blue_bot/description/robot.urdf.xacro`.

```xml
<?xml version="1.0"?>
<robot xmlns:xacro="http://www.ros.org/wiki/xacro"  name="robot">

    <!-- we'll add the robot_core.xacro -->
    <xacro:include filename="robot_core.xacro" />
    
</robot>
```

Now we'll add a new file `robot_core xacro` in the same directory.  

We also need to install joint_state_publisher_gui using

```bash
sudo apt install ros-jazzy-joint-state-publisher-gui
```

### `Links`

Each link tag will represent 1 link. It must have a name, but can also have 3 additional properties:
- <visual>
    - <geometry> defines the overall shape (box, cylinder, sphere, path to a 3d mesh)
    - <origin> offset for the geometry, so that it doesn't have to center around the origin.
    - <material> for color
</visual>

- <collision> for physics calculations, includes <geometry> and <origin> that can be copied from <visual>. But this can be made simple for computational savings.
    - <geometry> defines the overall shape (box, cylinder, sphere, path to a 3d mesh)
    - <origin> offset for the geometry, so that it doesn't have to center around the origin.
</collision>

- <inertial> also for physics calculation - how the links react to forces.
    - <mass>
    - <origin> the center of gravity, for most case the same as visual and collision origin.
    - <inertia> rotational inertia matrix. We can approximate our links as prisms, ellipsoids or cylinders.
</inertial>

- Seperate components, or
- Can move indepedently

**Link coordinate system**: if the `link` can rotate, it should have the origin at the pivot point. Example: 4 link coordinate system for: `Base`, `Slider`, `Arm`, `Camera`.

**Link joints**: joints show relation of the origin and coordinate frame of the links, which'll determine the position and orientation of the links.
It also shows what other link it's connected to - the `Parent`, and how it's connected to the `Parent`. Each link can have 1 parent, but many child links, each with own joints.
Example: a joint from `Base` to the `Slider`, one from the `Slider` to the `Arm`, and one from the `Arm` to the `Camera`.

**Link motion**:
- *Revolute*: rotational motion about a point, with fixed start and stop angle.
- *Continuous*: rotational motion about a point, without fixed limit - wheel.
- *Prismatic*: linear translational motion.
- *Fixed*: child link doesn't move relative to the parent link.

### Joints
Joints specify where the links are in space.
Parent and child links define links this joint 

```xml
<joint name="arm_joint" type="revolute">
    <parent link="slider_link"/>
    <child link="arm_link"/>
    <origin xyz="0.25 0 0.15" rpy="0 0 0"> relationship of the links before any motion is applied.
    For non-fixed joints, we need at least two more characteristics:
    <axis xyz="0 -1 0"/> which axes (combination of x, y, z) the joint moves along or aournd.
    <limits lower="0" upper="${pi/2}" velocity="100" effort="100"/> upper and lower positional limits in radians or meters, velocity limit in radians or meters per second, and effort limits in newtons or newton meters.
</joint>
```

Other tags include <material> tag for defining colors, <gazebo> tag for simulation, <transmission> tag that defines how an actuator is connected to a joint.

Naming convention: "_link", "_joint". Like "arm_link" is the child link in "arm_joint".

### xacro
XML macro (xacro) helps with urdf files.
- Splitting urdf into multiple files
- Avoidance of repetition

To add xacro in a URDF file:
<robot xmlns:xacro="http://www.ros.org/wiki/xacro">

xacro compiles split urdf files into one urdf, which is fed to robot_state_publisher, which publishes the complete urdf on /robot_description topic.

my_robot.urdf.xacro
'''xml
<?xml version="1.0"?>
<robot xmlns:xacro="http://www.ros.org/wiki/xacro" name="my_robot">
    <xacro:include filename="my_materials.xacro"/>
    <link>...</link>
    <joint>...</joint>
</robot>
'''

my_materials.xacro
'''xml
<?xml version="1.0"?>
<robot xmlns:xacro="http://www.ros.org/wiki/xacro">
    <material name="white">
        <color rgba="1 1 1 1"/>
    </material>
</robot>
'''

*Properties*: like setting variables.
<xacro:property name="arm_radius" value="0.5"/>
...
<cylinder radius="${arm_radius}" length="7"/>

*Math operations*:
<cylinder length="${4*arm_radius*pi1}"/>

*Macros*:
<xacro:macro name="inertial_box" params="mass x y z *origin">
    <inertial1>
        <xacro:insert_block name="origin"/>
        <mass value="${mass}"/>
        <inertia ixx="${(1/12) * mass * (y*y+z*z)}" *other entries skipped*/>
    </inertial>
</xacro:macro>

User later:
<xacro:inertial_box mass="12" x="2" y="3" z="4">
    <origin xyz="0 2 4" rpy="0 0 0"/>
</xacro:inertial_box>

will expand to:
<inertial>
    <origin xyz="0 2 4" rpy="0 0 0"/>
    <mass value="12"/>
    <inertia ixx="25"/>
</inertial>


### Robot URDF Example

```xml
<?xml version="1.0"?>
<robot xmlns:xacro="http://www.ros.org/wiki/xacro">

    <xacro:include filename="inertial_macros.xacro" />

    <!-- we'll define colors to use later-->
    <material name="white">
        <color rgba="1 1 1 1"/>
    </material>
    
    <material name="black">
        <color rgba="0 0 0 1"/>
    </material>

    <material name="orange">
        <color rgba="1 0.3 0.1 1"/>
    </material>

    <material name="blue">
        <color rgba="0.2 0.2 1 1"/>
    </material>

    <!-- It's common in ROS to define BASE_LINK first. We define it in the center of the wheel connection, and define the rest of the body in terms of that location -->
    <!-- BASE_LINK -->

    <link name="base_link">
    </link>

    <!-- CHASSIS LINK -->
    <!-- Joint links parent to child link, where we can define the origin location offset - in this case it'll be 0.1m behind the center of the wheel connection. -->
    <joint name="chassis_joint" type="fixed">
        <parent link="base_link"/>
        <child link="chassis"/>
        <origin xyz="0 0 0"/>
    </joint>

    <!-- Now we'll create the actual chassis link -->
    <link name="chassis">
        <visual>
        <!-- The geometry will center in joint origin. We want to align it to the wheel connector -->
            <origin xyz="0 0 0.0625"/>
            <geometry>
                <cylinder radius="0.08" length="0.125"/>
            </geometry>
            <material name="blue"/>
        </visual>
        <collision>
        <!-- The geometry will center in joint origin. We want to align it to the wheel connector -->
            <origin xyz="0 0 0.0625"/>
            <geometry>
                <cylinder radius="0.08" length="0.125"/>
            </geometry>
        </collision>
        <xacro:inertial_cylinder mass="2.0" length="0.125" radius="0.08">
            <origin xyz="0 0 0.0625" rpy="0 0 0"/>
        </xacro:inertial_cylinder>
    </link>

    <!-- LEFT WHEEL LINK -->
    <joint name="left_wheel_joint" type="continuous">
        <parent link="base_link"/>
        <child link="left_wheel"/>
        <origin xyz="0 0.08 0" rpy="-${pi/2} 0 0"/>
        <axis xyz="0 0 1"/>
    </joint>

    <link name="left_wheel">
        <visual>
            <geometry>
                <cylinder radius="0.0325" length="0.02"/>
                <material name="black"/>
            </geometry>
        </visual>
        <collision>
            <geometry>
                <cylinder radius="0.0325" length="0.02"/>
            </geometry>
        </collision>
        <xacro:inertial_cylinder mass="0.5" length="0.02" radius="0.0325">
            <origin xyz="0 0 0" rpy="0 0 0"/>
        </xacro:inertial_cylinder>
    </link>

    <!-- RIGHT WHEEL LINK -->
    <joint name="right_wheel_joint" type="continuous">
        <parent link="base_link"/>
        <child link="right_wheel"/>
        <origin xyz="0 -0.08 0" rpy="${pi/2} 0 0"/>
        <axis xyz="0 0 -1"/>
    </joint>

    <link name="right_wheel">
        <visual>
            <geometry>
                <cylinder radius="0.0325" length="0.02"/>
                <material name="black"/>
            </geometry>
        </visual>
        <collision>
            <geometry>
                <cylinder radius="0.0325" length="0.02"/>
            </geometry>
        </collision>
        <xacro:inertial_cylinder mass="0.5" length="0.02" radius="0.0325">
            <origin xyz="0 0 0" rpy="0 0 0"/>
        </xacro:inertial_cylinder>
    </link>


    <!-- Front Caster Wheel Link -->
    <joint name="front_caster_wheel_joint" type="fixed">
        <parent link="chassis"/>
        <child link="front_caster_wheel"/>
        <origin xyz="0.07 0 0"/>
    </joint>

    <link name="front_caster_wheel">
        <visual>
            <geometry>
                <sphere radius="0.0325"/>
            </geometry>
            <material name="black"/>
        </visual>
        <collision>
            <geometry>
                <sphere radius="0.0325"/>
            </geometry>
        </collision>
        <xacro:inertial_sphere mass="0.5" radius="0.0325">
            <origin xyz="0 0 0" rpy="0 0 0"/>
        </xacro:inertial_sphere>
    </link>

    <!-- Back Caster Wheel Link -->
    <joint name="back_caster_wheel_joint" type="fixed">
        <parent link="chassis"/>
        <child link="back_caster_wheel"/>
        <origin xyz="-0.07 0 0"/>
    </joint>

    <link name="back_caster_wheel">
        <visual>
            <geometry>
                <sphere radius="0.0325"/>
            </geometry>
            <material name="black"/>
        </visual>
        <collision>
            <geometry>
                <sphere radius="0.0325"/>
            </geometry>
        </collision>
        <xacro:inertial_sphere mass="0.5" radius="0.0325">
            <origin xyz="0 0 0" rpy="0 0 0"/>
        </xacro:inertial_sphere>
    </link>

</robot>
```
After adding wheel link, run `ros2 run joint_state_publisher_gui joint_state_publisher_gui` before opening rviz2.

In rviz, set `Global Options > Fixed Frame`  to `base_link`. Add `/tf`, show names. Add robot body and set `Description Topic` to `/robot_description`.

Lastly, we'll add collisions - it's just duplicating visuals without materials.

Lastly, we'll include the inertial file and add intertial tags, run colcon build again.

We can save rviz config and later use `rviz2 -d src/blue_bot/config/blue_bot.rviz`.


## Driving the robot (in old Gazebo)


Robot launcher
```bash
ros2 launch blue_bot rsp.launch.py use_sim_time:=true # use_sim_time:=true synchronizes clock with Gazebo
```

Gazebo launcher
```bash
ros2 launch gazebo_ros gazebo.launch.py
```

Entity spawner in gazebo
```bash
ros2 run gazebo_ros spawn_entity.py -topic robot_description -entity <NAME> # URDF is published by robot_state_publisher on a topic called robot_description
```
Combining these 3 with a single launcher file:
```python

def generate_launch_description():


    # Include the robot_state_publisher launch file, provided by our own package. Force sim time to be enabled
    # !!! MAKE SURE YOU SET THE PACKAGE NAME CORRECTLY !!!

    package_name='articubot_one' #<--- CHANGE ME
    
    rsp = IncludeLaunchDescription(
                PythonLaunchDescriptionSource([os.path.join(
                    get_package_share_directory(package_name),'launch','rsp.launch.py'
                )]), launch_arguments={'use_sim_time': 'true', 'use_ros2_control': 'true'}.items()
    )

    # Include the Gazebo launch file, provided by the gazebo_ros package
    gazebo_params_file = os.path.join(get_package_share_directory(package_name),'config','gazebo_params.yaml')

    gazebo = IncludeLaunchDescription(
                PythonLaunchDescriptionSource([os.path.join(
                    get_package_share_directory('gazebo_ros'), 'launch', 'gazebo.launch.py')]),
                    launch_arguments={'extra_gazebo_args': '--ros-args --params-file ' + gazebo_params_file}.items()
             )

    # Run the spawner node from the gazebo_ros package. The entity name (my_bot here) doesn't really matter if you only have a single robot.
    spawn_entity = Node(package='gazebo_ros', executable='spawn_entity.py',
                        arguments=['-topic', 'robot_description',
                                   '-entity', 'my_bot'],
                        output='screen')


    diff_drive_spawner = Node(
        package="controller_manager",
        executable="spawner",
        arguments=["diff_cont"],
    )

    joint_broad_spawner = Node(
        package="controller_manager",
        executable="spawner",
        arguments=["joint_broad"],
    )

    # Launch them all!
    return LaunchDescription([
        rsp,
        joystick,
        twist_mux,
        gazebo,
        spawn_entity,
        diff_drive_spawner,
        joint_broad_spawner
    ])

```

Now launch using the combined launcher
```bash
ros2 run gazebo_ros launch_sim.launch.py
```

![Robot Control](./images/robot_control_1.png)

For a differential drive robot, we just need `linear.x` and `angular.z`.  
Getting position/odometry from true velocity (which comes from motor speed measurements) is called dead reckoning.  
We use plugins for contol system (gazebo_ros_diff_drive or ros2_control). This system fits with the overall robot state.

![Robot State Publisher and Control](./images/statePublisher_control.png)

`/joint_states` are published by the `Control Plugin`. `odom` is the start position, and a transform is broadcasted from `odom->base_link`, that let's us know the current position estimate of the robot.

## Robot brain: pi
* PC
    * 64-bit
    * RAM: >= 4GB
    * Storage: >=
    * 5V low current
* OS
    * ROS compatible Ubuntu
* Software
    * Git
    * Openssh-server
    * Arduino IDE
    * VS Code
* Networking: remote PC and pi on the same network with internet
* ROS
    * Install main packages following official instructions
    * Install colon
    * Setup environment
* Setup workspace and packages


## Powering the robot
| *Name* | *Other Names*        | *Units*              | *Symbol in Equations* | *Water Analogy*       |
|------------------|----------------------|----------------------|------------------------|------------------------|
| Voltage          | Potential Difference | Volts (V)            | V                      | Water Pressure         |
| Current          | Amperage             | Amperes/Amps (A, mA) | I                      | Flow Rate of Water     |
| Power            | Wattage              | Watts (W)            | P                      | Power of Flow (Turbine)|

* $P=VI$. Power in supplier indicates the max power it can provide (5W with 5V-1A) or draw.

* Most electrical components use 5V
* Motors draw more
* We use switching regulator/buck converter to convert high voltage into 5V.

### Deciding battery voltage
12V for DC motors

### Current draw
![Estimated current draw](./images/current_draw.png)

* The demand is about 5A. So the regulator needs to be capable of delivering $5V\times 5A=25W$ of power.
* The supply is $25W/12V\approx 2.1\rightarrow 2.5A$
* Motor demand: highest (stall) current draw * 2 $1.8A\times 2=3.6A$
* Total current $2.5+3.6=6.1A$

### Battery
LiPo
* S number: number of cells
* Cell voltage: nominally 3.7V (11.1 for 3S); safe range 3.2V-4.2V (**3S 9.6V-12.6V**)
* Capacity: how much mA it takes to empty the battery in 1 hour - 3000 mAh battery could run for 30 minutes at 6A. But you'd want to use at most 2/3.
* C number: max discharge current = C number $\times$ capacity. 3000 mAh $\times$ 20C = 60A max current (draining the battery in 3 minutes!).

LiPo needs careful handeling.
* Check voltage regularly
* Charge with a good charger at a fireproof setup
* Make sure the wiring (gauge) and connector current carrying capacity. For example, breadboards can only handle milliamps.
* Pi can handle specific current flow, so external devices should be powered seperately, with an USB hub for instance.
* We also need switch and fuse for safety.

![Power circuit](./images/power_circuit.png)



## Motors
* Robot controller (e.g., Pi calculating required speed)
  * Comms layer: computer to motor controller (e.g., USB serial)
    * closed-loop or feedback control (e.g., PID control takes motor measurement and adjusts target speed)
      * open-loop control (map joystick button to target speed)
        * motor controller (takes target-speed/position, outs low-voltage signal)
          * motor driver (takes low-voltage signal and current, outs amplified high-voltage signal)
            * voltage signal
              * 12V DC motor

![Motor Circuit](./images/motor_loop.png)

### L298N motor driver
* It has 12V in for motors, and 5V in for the L298N chip.
* Pair of terminals in each sides are for the motors.

### Motor-Driver-Arduino Pairing
* Upload driver code on the Arduino
* Connect Arduino to the motor driver
* Check if the motor control works using miniterm (by sending serial command)
    * e for checking current speed
    * r to reset
    * o (+/-)0-255 (+/-)0-255 for spinning at variable speed with open-loop control (the wheels may rotate at different rates for same speed)
    * m (+/-)0-255 (+/-)0-255 for spinning at variable speed with closed-loop control

* The motor encoders can have 4 wires - two will go to 5V and Gnd to power the encoder circuit, two will go to Arduino input.
* After connecting the encoder to the Arduino, check if rotating the wheel gives positive return (r in miniterm). Otherwise, swap the motor power wires.
* Rotate the wheel by n times, get encoder reading e (e in miniterm), e/n is rev of encoders per rev of wheel.
* In the arduino code, PID loop runs 30 loops/sec, number of encoder counts per PID loop = (e/n)/30 - is the magnitude (range 0-255) for 1 rev/sec.

### ROS driver
Demo driver with two nodes, an encoder listener in topic and send them to the controller, another gui for sending commands.

On the PI:
* Clone https://github.com/joshnewans/serial_motor_demo in workspace (src) folder, build, source.

```bash
ros2 run serial_motor_demo driver --ros-args -p serial_port:=/dev/ttyUSB0 -p baud_rate:=57600 -p loop_rate:=30 -p encoder_cpr:=3450 # ros2 run <PACKAG> <NODE> 
```
`Error: Serial timeout on command: e` is OK. 

On the dev machine:
```bash
ros2 run serial_motor_demo gui
```


## Sensors
### LiDAR
Can be based on sound or light, 1/2/3D.

In ROS, different LiDAR are subscribed using `/scan`.

In the robot description `robot.urdf.xacro`, we'll add another xacro file for lidar.

In the LiDAR xacro, we'll start with a joint and a link.

```xml
<?xml version="1.0"?>
<robot xmlns:xacro="http://www.ros.org/wiki/xacro" >

    <joint name="laser_joint" type="fixed">
        <parent link="chassis"/>
        <child link="laser_frame"/>
        <origin xyz="0.122 0 0.212" rpy="0 0 0"/>
    </joint>

    <link name="laser_frame">
        <visual>
            <geometry>
                <cylinder radius="0.05" length="0.04"/>
            </geometry>
            <material name="black"/>
        </visual>
        <visual>
            <origin xyz="0 0 -0.05"/>
            <geometry>
                <cylinder radius="0.01" length="0.1"/>
            </geometry>
            <material name="black"/>
        </visual>
        <collision>
            <geometry>
                <cylinder radius="0.05" length="0.04"/>
            </geometry>
        </collision>
        <xacro:inertial_cylinder mass="0.1" length="0.04" radius="0.05">
            <origin xyz="0 0 0" rpy="0 0 0"/>
        </xacro:inertial_cylinder>
    </link>



    <gazebo reference="laser_frame">
        <material>Gazebo/Black</material>

        <sensor name="laser" type="ray">
            <pose> 0 0 0 0 0 0 </pose>
            <visualize>false</visualize>
            <update_rate>10</update_rate>
            <ray>
                <scan>
                    <horizontal>
                        <samples>360</samples>
                        <min_angle>-3.14</min_angle>
                        <max_angle>3.14</max_angle>
                    </horizontal>
                </scan>
                <range>
                    <min>0.3</min>
                    <max>12</max>
                </range>
            </ray>
            <plugin name="laser_controller" filename="libgazebo_ros_ray_sensor.so">
                <ros>
                    <argument>~/out:=scan</argument>
                </ros>
                <output_type>sensor_msgs/LaserScan</output_type>
                <frame_name>laser_frame</frame_name>
            </plugin>
        </sensor>
    </gazebo>

</robot>
```
In Rviz2, we can add LaserScan topic.

#### Connecting physical LiDAR
On Pi, install driver with `sudo apt install` command. Then,

In order to figure out the USB where the LiDAR is connected, run `ls /dev/serial/by-path/` and double tab to get the name.

```bash
ros2 rplidar_ros rplidar_composition --ros-args -p serial_port:=/dev/serial/by-path/<NAME> -p frame_id:=laser_frame -p angle_compensate:=true -p scan_mode:=Standard
```

Now on dev machine, launch `rviz2`. If the robot state publisher is not published, select Fixed Frame `laser_frame`.

For starting/ending LiDAR motor, run `ros2 service call` and double tap for completion, then select `/stop_motor`, then double tap to get `std_srvs/srv/Empty`, finally add an empty message `{}`. Similar for `/start_motor`.

```bash
ros2 service call /stop_motor std_srvs/srv/Empty {}
```

We can add a launch file with all parameters:
```python
import os
from launch import LaunchDescription
from launch_ros.actions import Node

def generate_launch_description():

    return LaunchDescription([

        Node(
            package='rplidar_ros',
            executable='rplidar_composition',
            output='screen',
            parameters=[{
                'serial_port': '/dev/serial/by-path/platform-fd500000.pcie-pci-0000:01:00.0-usb-0:1.3:1.0-port0',
                'frame_id': 'laser_frame',
                'angle_compensate': True,
                'scan_mode': 'Standard'
            }]
        )
    ])
```

And launch rplidar using
```bash
ros2 launch rplidar.launch.py
```

If this hangs the Pi, connect to Pi in another terminal with ssh, and run `killall rplidar_composition`.



### Camera
#### Camera and images fundamentals
* There are many different types of cameras
  * RGB, infrared, thermal
  * Wide angle, telephoto
* Capture light through a lense, apparture, onto a sensor.

##### Image representation
Stored in 2D array of pixels. For RGB, have 3 color channels for each pixel - RGB, store intensity of each in with 1-byte (8-bits) per color channel per pixel 0-255.

##### Image compression
For better data transfer, the images are compressed. PNG is a lossless compression, JPEG is lossy. It takes processing, but the bandwidth gain is worth it.

##### Camera parameter - Focal length
Distance of lense and sensor - affecting horizontal Field of View (FOV). Increasing focal length decreases FOV, resulting in a zoomed in image. $\text{h\_fov}=2\tan^{-1}\left(\frac{\text{sensor\_width}}{2\times \text{focal\_length}}\right)$

##### Coordinate system
Following right-hand rule
* x-direction: left to right
* y-direction: top to bottom
* z-direction: away from the camera

#### ROS-Camera/image integration

#### Gazebo camera simulation

#### Connecting a real camera and getting data

```bash
ros2 run rqt_image_view rqt_image_view

ros2 run image_transport republish compressed raw --ros-args -r in/compressed:=/camera/image_raw/compressed -r out:=/camera/image_raw/uncompressed
```

### Depth camera



## Chassis design

## Ros2 Control


### Driving using Ros2 control

### Wireless control

### Phone control

## SLAM

## Nav2

## Additional hardware

## Humble

## New Gazebo

## Camera
